# 사전 준비

구글 드라이브와 연동하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


라이브러리 설치

In [ ]:
!pip install torch==1.3.1
!pip install transformers==2.1.1
!pip install tqdm
!pip install sentencepiece==0.1.85
!pip install pandas 

     |████████████████████████████████| 734.6MB 23kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.3.1 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.3.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
     |████████████████████████████████| 317kB 8.2MB/s 
     |████████████████████████████████| 102kB 12.4MB/s 
     |████████████████████████████████| 901kB 34.7MB/s 
     |████████████████████████████████| 1.2MB 48.2MB/s 
     |████████████████████████████████| 7.6MB 45.2MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
  Created wheel for boto3: filename=boto3-1.17.78-py2.py3-none-any.whl size=128921 sha256=339b7445e2a0fe4374c599ab0bbde6ca606ad5adcd0617cad8720348569266c5
  Stored in directory: /root/.cache/pip/wheels/35/4e/0b/65ce88240e97b158d4a062630f6815337a79cd64bbc61bbd1

In [ ]:
!pip install gluonnlp 

     |████████████████████████████████| 348kB 8.0MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595676 sha256=af1e05936dbb4fc943011b175c363f092fea913f8c61a96aa2a2b031d9b11365
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp


In [ ]:
!pip install mxnet-cu101

     |████████████████████████████████| 356.7MB 45kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-227_i2zp
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-227_i2zp
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=d6c1be7bdd69f38fd591692c140b077ad95e17ebc1fadec1a3f29218f773e774
  Stored in directory: /tmp/pip-ephem-wheel-cache-kd3hm8ge/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


# 필요한 라이브러리 가져오기

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule
import pandas as pd

# GPU 사용 가능한지 확인

In [ ]:
device = torch.device("cuda:0")

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

# 사전 훈련된 kobert 단어 사전(vocab)과 모델 가져오기

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
vocab

# 토크나이저 만들기 및 파라미터 설정하기

토크나이저 만들기

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


데이터셋을 버트 모델에 맞게 토크나이징 하기

In [ ]:

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([str(i[sent_idx])]) for i in dataset.values]
        self.labels = [np.int32(i[label_idx]) for i in dataset.values]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


파라미터 설정하기

In [ ]:
## Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
transform = nlp.data.BERTSentenceTransform(
            tok, max_seq_length=max_len, pad=True, pair=False)

# 데이터셋 준비하기

데이터셋 불러오기(유튜브 데이터셋은 따로 유튜브 댓글을 크롤링 하여 직접 악플인지 아닌지 라벨링 하여 만든 데이터셋)(이 코드를 이용할 경우 준비한 데이터셋을 따로 사용할 것)

In [ ]:
import pandas as pd
path="/content/drive/MyDrive/Colab Notebooks/Comment/"

유튜브 전체 데이터셋

In [ ]:
dataset_train = pd.read_csv(path + "merged_train.csv", encoding = "cp949")
dataset_test = pd.read_csv(path + "merged_test.csv", encoding = "cp949")

유튜브 악플, 일반 댓글 5:5 비율 데이터셋

In [ ]:
dataset_train = pd.read_csv(path + "merged_5ratio_train.csv", encoding = "cp949")
dataset_test = pd.read_csv(path + "merged_5ratio_test.csv", encoding = "cp949")

유튜브 악플, 일반 댓글 2.5:7.5 비율 데이터셋

In [ ]:
dataset_train = pd.read_csv(path + "merged_75ratio_train.csv", encoding = "cp949")
dataset_test = pd.read_csv(path + "merged_75ratio_test.csv", encoding = "cp949")

네이버 무비 리뷰 데이터셋

In [ ]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

데이터셋 전처리

In [ ]:
dataset_train.Comment = dataset_train.Comment.astype(str)
dataset_test.Comment = dataset_test.Comment.astype(str)

In [ ]:
dataset_train.Comment = dataset_train.Comment.apply(lambda x : x if len(x) <= 140 else np.nan)
dataset_test.Comment = dataset_test.Comment.apply(lambda x : x if len(x) <= 140 else np.nan)
dataset_train = dataset_train.dropna()
dataset_test = dataset_test.dropna()

In [ ]:
dataset_train.Target = dataset_train.Target.astype(np.int32)
dataset_test.Target = dataset_test.Target.astype(np.int32)

데이터셋 크기 확인하기

In [ ]:
print(dataset_test.shape)
print(dataset_train.shape)

(2039, 2)
(6115, 2)


준비된 데이터셋을 BERT에 맞게 변환하기

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

데이터셋을 배치사이즈에 맞게 변환해주기

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

# 모델 만들기

분류기 만들기

In [ ]:
class KOBERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(KOBERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = KOBERTClassifier(bertmodel,  dr_rate=0.5).to(device)

손실 함수, 옵티마이저, 가중치 감소, 웜업,  학습률 스케쥴러 설정하기  

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

 웜업, 학습률 스케쥴러 설정하기

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

정확도 계산 함수 만들기

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 모델 훈련시키기

GPU 캐시 초기화하기

In [ ]:
torch.cuda.empty_cache()

모델 학습시키기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

NameError: ignored

In [ ]:
def convert_sentence(sentence, tokenizer = tok, max_seq_length = max_len, pad =True, pair = False):
  transform = nlp.data.BERTSentenceTransform(
            tokenizer, max_seq_length, pad, pair)
  tokenized_text =  [transform(str([sentence]))]
  return tokenized_text

In [ ]:
def make_sentence(word, Target):
  sentence = pd.DataFrame({"Comment": [str(word)],"Target":[int(Target)]})
  return sentence

모델 저장 경로 설정하기

In [ ]:
# path="/content/drive/MyDrive/Colab Notebooks/ML_model_youtube_75ratio.pt"

모델 저장하기

In [ ]:
# torch.save(model.state_dict(),path)

# 모델 검증하기

모델 불러오기

In [ ]:
path="/content/drive/MyDrive/Colab Notebooks/ML_model_new.pt"
model = model.load_state_dict(torch.load(path))

데이터셋 형태로 문장 변환하는 함수 만들기

In [ ]:
def make_sentence(word, Target):
  sentence = pd.DataFrame({"Comment": [str(word)],"Target":[int(Target)]})
  return sentence

입력받은 데이터셋 형태의 문장 BERT에 맞게 변환하는 함수 만들기

In [ ]:
def convert_sentence(sentence, tokenizer = tok, max_seq_length = max_len, pad =True, pair = False):
  transform = nlp.data.BERTSentenceTransform(
            tokenizer, max_seq_length, pad, pair)
  tokenized_text =  [transform(str([sentence]))]
  return tokenized_text

문장 입력하기

In [ ]:
sentence = make_sentence("랩 접고 기술이나 배워라 ㅋㅋㅋㅋ 답이 없다 너는ㅋㅋㅋ", 1)
data_sentence = BERTDataset(sentence, 0, 1, tok, max_len, True, False)
sentence_dataloader = torch.utils.data.DataLoader(data_sentence, batch_size=32, num_workers=0, shuffle=True)

분류 결과 확인하기

In [ ]:
model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(sentence_dataloader):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        logit = out
        # print(logit)
        logit = logit.detach().cpu().numpy()
        state = np.argmax(logit)
        if state == 1:
          state = "Yes"
        else :
          state = "No"
        print("악플인가? : {state}".format(state = state))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


악플인가? : No



# 모델들 검증하기

데이터셋 준비하기 및 모델 만들기를 스킵 하였을 시 아래 4개의 셀을 실행

In [ ]:
class KOBERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(KOBERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
import pandas as pd
path="/content/drive/MyDrive/Colab Notebooks/Comment/"

모델들 불러오기

In [ ]:
path="/content/drive/MyDrive/Colab Notebooks/ML_model_new.pt"
model_naver = KOBERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model_naver.load_state_dict(torch.load(path))

path="/content/drive/MyDrive/Colab Notebooks/ML_model_youtube_original.pt"
model_youtube_original = KOBERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model_youtube_original.load_state_dict(torch.load(path))

path="/content/drive/MyDrive/Colab Notebooks/ML_model_youtube_5ratio.pt"
model_youtube_5ratio = KOBERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model_youtube_5ratio.load_state_dict(torch.load(path))

path="/content/drive/MyDrive/Colab Notebooks/ML_model_youtube_75ratio.pt"
model_youtube_75ratio = KOBERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model_youtube_75ratio.load_state_dict(torch.load(path))

<All keys matched successfully>

모델들 비교하는 함수 만들기

In [ ]:
def compare(models, sentence_dataloader):
    results = []
    i = 0
    for model in models:
        model.eval()
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(sentence_dataloader):
            optimizer.zero_grad()
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            logit = out
            # print(logit)
            logit = logit.detach().cpu().numpy()
            state = np.argmax(logit)
            if i == 0:
                name = "model_naver"
                if state == 1:
                    state = "No"
                else :
                    state = "Yes"
            elif i == 1:
                name = "model_youtube_original"
                if state == 1:
                    state = "Yes"
                else :
                    state = "No"
            elif i == 2:
                name = "model_youtube_5ratio"
                if state == 1:
                    state = "Yes"
                else :
                    state = "No"
            elif i == 3:
                name = "model_youtube_75ratio"
                if state == 1:
                    state = "Yes"
                else :
                    state = "No"
            i+=1
            results.append([name, state])
    print("악플인가?")
    for result in results:
        print("{model} : {state}".format(state = result[1],model = result[0]))

모델들 분류 결과 비교하기

In [ ]:
sentence = make_sentence("너는 기술이나 배워라 ㅋㅋㅋ 재능이 없다", 1)
data_sentence = BERTDataset(sentence, 0, 1, tok, max_len, True, False)
sentence_dataloader = torch.utils.data.DataLoader(data_sentence, batch_size=32, num_workers=0, shuffle=True)


models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare(models, sentence_dataloader)

악플인가?
model_naver : No
model_youtube_original : No
model_youtube_5ratio : Yes
model_youtube_75ratio : No


In [ ]:
sentence = make_sentence("멍청한게 자랑인가 ㅋㅋㅋㅋ", 1)
data_sentence = BERTDataset(sentence, 0, 1, tok, max_len, True, False)
sentence_dataloader = torch.utils.data.DataLoader(data_sentence, batch_size=32, num_workers=0, shuffle=True)


models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare(models, sentence_dataloader)

악플인가?
model_naver : Yes
model_youtube_original : Yes
model_youtube_5ratio : Yes
model_youtube_75ratio : Yes


In [ ]:
sentence = make_sentence("ㅁㅊ놈", 1)
data_sentence = BERTDataset(sentence, 0, 1, tok, max_len, True, False)
sentence_dataloader = torch.utils.data.DataLoader(data_sentence, batch_size=32, num_workers=0, shuffle=True)


models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare(models, sentence_dataloader)

악플인가?
model_naver : Yes
model_youtube_original : Yes
model_youtube_5ratio : Yes
model_youtube_75ratio : Yes


In [ ]:
sentence = make_sentence("ㅁㅊ놈 ㅈㄴ웃기네ㅋㅋㅋ ", 0)
data_sentence = BERTDataset(sentence, 0, 1, tok, max_len, True, False)
sentence_dataloader = torch.utils.data.DataLoader(data_sentence, batch_size=32, num_workers=0, shuffle=True)


models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare(models, sentence_dataloader)

악플인가?
model_naver : No
model_youtube_original : No
model_youtube_5ratio : No
model_youtube_75ratio : No


In [ ]:
sentence = make_sentence("노력하는 모습이 멋있다.", 0)
data_sentence = BERTDataset(sentence, 0, 1, tok, max_len, True, False)
sentence_dataloader = torch.utils.data.DataLoader(data_sentence, batch_size=32, num_workers=0, shuffle=True)


models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare(models, sentence_dataloader)

악플인가?
model_naver : No
model_youtube_original : No
model_youtube_5ratio : No
model_youtube_75ratio : No


In [ ]:
sentence = make_sentence("뭐라는 거야ㅋㅋㅋ 넌 그냥 역겨워 착각하지마", 1)
data_sentence = BERTDataset(sentence, 0, 1, tok, max_len, True, False)
sentence_dataloader = torch.utils.data.DataLoader(data_sentence, batch_size=32, num_workers=0, shuffle=True)


models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare(models, sentence_dataloader)

악플인가?
model_naver : Yes
model_youtube_original : No
model_youtube_5ratio : Yes
model_youtube_75ratio : Yes


In [ ]:
sentence = make_sentence("공주병 말기네ㅋㅋㅋ 나잇값 좀 하세요 ^^", 1)
data_sentence = BERTDataset(sentence, 0, 1, tok, max_len, True, False)
sentence_dataloader = torch.utils.data.DataLoader(data_sentence, batch_size=32, num_workers=0, shuffle=True)


models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare(models, sentence_dataloader)

악플인가?
model_naver : No
model_youtube_original : No
model_youtube_5ratio : No
model_youtube_75ratio : No


In [ ]:
sentence = make_sentence("미안한데 넌 영원히 안돼 포기해 ㅋㅋㅋㅋ", 1)
data_sentence = BERTDataset(sentence, 0, 1, tok, max_len, True, False)
sentence_dataloader = torch.utils.data.DataLoader(data_sentence, batch_size=32, num_workers=0, shuffle=True)


models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare(models, sentence_dataloader)

악플인가?
model_naver : No
model_youtube_original : No
model_youtube_5ratio : No
model_youtube_75ratio : No


모델들 정확도 비교하는 함수 만들기

In [ ]:
def compare_acc(models,test_dataloader):
    results = []
    i = 0
    for model in models:
        model.eval()
        test_acc = 0.0
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
            optimizer.zero_grad()
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            test_acc += calc_accuracy(out, label)
            if i == 0:
                name = "model_naver"
            elif i == 1:
                name = "model_youtube_original"
            elif i == 2:
                name = "model_youtube_5ratio"
            elif i == 3:
                name = "model_youtube_75ratio"
        acc =  test_acc / (batch_id+1)
        if i == 0:
          acc = 1 - acc
        i+=1
        results.append([name, acc])
    print("정확도")
    for result in results:
        print("{model} : {acc}".format(acc = result[1],model = result[0]))

모델들 정확도 비교하기

유튜브 전체 데이터셋 테스트

In [ ]:
dataset_test = pd.read_csv( "/content/drive/MyDrive/Colab Notebooks/Comment/merged_test.csv", encoding = "cp949")
dataset_test.Comment = dataset_test.Comment.astype(str)
dataset_test.Comment = dataset_test.Comment.apply(lambda x : x if len(x) <= 140 else np.nan)
dataset_test = dataset_test.dropna()
dataset_test.Target = dataset_test.Target.astype(np.int32)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare_acc(models, test_dataloader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



정확도
model_naver : 0.7638257575757575
model_youtube_original : 0.9343181818181818
model_youtube_5ratio : 0.8923484848484848
model_youtube_75ratio : 0.8992045454545454


유튜브 악플, 일반 댓글 5:5 비율 데이터셋 테스트

In [ ]:
dataset_test = pd.read_csv( "/content/drive/MyDrive/Colab Notebooks/Comment/merged_5ratio_test.csv", encoding = "cp949")
dataset_test.Comment = dataset_test.Comment.astype(str)
dataset_test.Comment = dataset_test.Comment.apply(lambda x : x if len(x) <= 140 else np.nan)
dataset_test = dataset_test.dropna()
dataset_test.Target = dataset_test.Target.astype(np.int32)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare_acc(models, test_dataloader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



정확도
model_naver : 0.7790625
model_youtube_original : 0.5689062500000001
model_youtube_5ratio : 0.7759375000000001
model_youtube_75ratio : 0.76890625


유튜브 악플, 일반 댓글 2.5:7.5 비율 데이터셋

In [ ]:
dataset_test = pd.read_csv( "/content/drive/MyDrive/Colab Notebooks/Comment/merged_75ratio_test.csv", encoding = "cp949")
dataset_test.Comment = dataset_test.Comment.astype(str)
dataset_test.Comment = dataset_test.Comment.apply(lambda x : x if len(x) <= 140 else np.nan)
dataset_test = dataset_test.dropna()
dataset_test.Target = dataset_test.Target.astype(np.int32)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

models = [model_naver, model_youtube_original, model_youtube_5ratio, model_youtube_75ratio]
compare_acc(models, test_dataloader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



정확도
model_naver : 0.6873301630434783
model_youtube_original : 0.7936693274456522
model_youtube_5ratio : 0.798934273097826
model_youtube_75ratio : 0.7991465692934783


# 결과

총 3개의 테스트 데이터 집합에 모델 정확도를 검증하고 이를 종합해 본 결과, 수치상으로는
1. model_youtube_75ratio
2. model_youtube_5ratio
3. model_youtube_original
4. model_naver

순으로 우수하지만 새로운 문장에 대한 테스트에서는 model_youtube_5ratio의 성능이 가장 우수하였다. 단순히 데이터의 양을 많이 하는 것 보다, 악플과 일반 댓글의 비율을 맞춰주는 것이 데이터의 양이 훨씬 적을지라도 더 높은 성능을 낼 수 있음을 확인할 수 있었다. 높은 성능의 모델을 만들기 위해서는 단순한 데이터 수집만이 아닌, 전처리 방식이나, 데이터 내에서의 레이블에 따른 데이터 비율 등등을 고려해줘야 함을 느꼈다. 또한 신기한 것은 의외로 네이버 무비 리뷰 데이터셋으로 학습한 모델이 유튜브 악플들을 생각보다 잘 잡는 것이었다. 데이터 양도 많고 그 비율도 5:5 비율로 유지되서 다른 도메인에서도 성능을 잘 내는 것이라고 생각한다. 만약 유튜브 악플 데이터가 지금 보다 훨씬 늘어 1만 개, 2만 개 정도 있었다면 훨씬 좋은 성능의 모델이 나왔을 거라 생각한다. 하지만 그만큼 데이터 수집과 레이블링, 전처리에 많은 시간이 투자될 것이다. 머신러닝, 딥러닝 프로젝트는 모델 학습 보다도 양질의 데이터 수집과 전처리 과정이 더 힘들다고 느껴진다.
